# Sakila EDA - DuckDB + Pandas

In [1]:
import duckdb

conn = duckdb.connect("../data/sakila.duckdb")
conn.sql("SET search_path='staging';")
describe = conn.sql("desc;").df()

In [2]:
describe

,database,schema,name,column_names,column_types,temporary
0,sakila,staging,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,sakila,staging,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,sakila,staging,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,sakila,staging,actor,"[actor_id, first_name, last_name, last_update,...","[DECIMAL(38,9), VARCHAR, VARCHAR, TIMESTAMP, V...",False
4,sakila,staging,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
5,sakila,staging,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
6,sakila,staging,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",False
7,sakila,staging,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
8,sakila,staging,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
9,sakila,staging,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False


### a) Which movies are longer than three hours? (Showing title and length)

In [3]:
conn.sql("""--sql
SELECT
    title,
    length
FROM
    film
WHERE
    length > 180
ORDER BY
    length DESC;
""").df().head(10)

,title,length
0,WORST BANGER,185
1,CHICAGO NORTH,185
2,CONTROL ANTHEM,185
3,DARN FORRESTER,185
4,SWEET BROTHERHOOD,185
5,GANGS PRIDE,185
6,HOME PITY,185
7,SOLDIERS EVOLUTION,185
8,POND SEATTLE,185
9,MUSCLE BRIGHT,185


### b) Which movies have the word "love" in its title? (Showing title, rating, length, description)

In [4]:
conn.sql("""--sql
SELECT
	title,
    rating,
    length,
    description
FROM
    film
WHERE
    title ILIKE '%love%';
""").df()

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


### c) Longest, shortest, median and average length movies

In [5]:
conn.sql("""--sql
SELECT
    MIN(length) || ' ' || 'minutes' as shortest_movie,
    MAX(length) || ' ' || 'minutes'  as longest_movie,
    AVG(length)::INT || ' ' || 'minutes'  as average_movie_length,
    MEDIAN(length)::INT  || ' ' || 'minutes' as median_movie_length
FROM
    film
""").df()

,shortest_movie,longest_movie,average_movie_length,median_movie_length
0,46 minutes,185 minutes,115 minutes,114 minutes


### d) 10 most expensive movie to rent per day

In [6]:
rent_film = conn.sql("""--sql
SELECT
    title,
    release_year,
    description,
    (rental_rate / rental_duration) AS rent_per_day
FROM
    film
ORDER BY
    rent_per_day DESC;
""").df()

rent_film.head(10)

,title,release_year,description,rent_per_day
0,PATHS CONTROL,2006,A Astounding Documentary of a Butler And a Cat...,1.663333
1,BACKLASH UNDEFEATED,2006,A Stunning Character Study of a Mad Scientist ...,1.663333
2,BEHAVIOR RUNAWAY,2006,A Unbelieveable Drama of a Student And a Husba...,1.663333
3,VIRTUAL SPOILERS,2006,A Fateful Tale of a Database Administrator And...,1.663333
4,TEEN APOLLO,2006,A Awe-Inspiring Drama of a Dog And a Man who m...,1.663333
5,TELEGRAPH VOYAGE,2006,A Fateful Yarn of a Husband And a Dog who must...,1.663333
6,BILKO ANONYMOUS,2006,A Emotional Reflection of a Teacher And a Man ...,1.663333
7,HEARTBREAKERS BRIGHT,2006,A Awe-Inspiring Documentary of a A Shark And a...,1.663333
8,CLERKS ANGELS,2006,A Thrilling Display of a Sumo Wrestler And a G...,1.663333
9,TIES HUNGER,2006,A Insightful Saga of a Astronaut And a Explore...,1.663333


### e) Actor who played in most movies

In [7]:
actors_in_movies = conn.sql("""--sql
SELECT
    a.first_name || ' ' || a.last_name AS actor_name,
    f.title,
FROM
    actor a
    LEFT JOIN film_actor fa ON fa.actor_id = a.actor_id
    LEFT JOIN film f ON f.film_id = fa.film_id;
""").df()

actors_in_movies["actor_name"].value_counts().head(5)

actor_name
SUSAN DAVIS       54
GINA DEGENERES    42
WALTER TORN       41
MARY KEITEL       40
MATTHEW CARREY    39
Name: count, dtype: int64

### f) Adding some interesting insights on my own (with a little help from my friend ChatGPT)

#### f.1) What movie categories are most rented?

In [8]:
conn.sql("""--sql
SELECT
    c.name AS category,
    COUNT(r.rental_id) AS total_rentals
FROM
    category c
    JOIN film_category fc ON fc.category_id = c.category_id
    JOIN film f ON f.film_id = fc.film_id
    JOIN inventory i ON f.film_id = i.film_id
    JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY
    c.name
ORDER BY
    total_rentals DESC
LIMIT
    10;
""").df()


,category,total_rentals
0,Sports,1179
1,Animation,1166
2,Action,1112
3,Sci-Fi,1101
4,Family,1096
5,Drama,1060
6,Documentary,1050
7,Foreign,1033
8,Games,969
9,Children,945


#### f.2) Which customers are most active overall and per month?

In [9]:
conn.sql("""--sql
SELECT
    cu.first_name || ' ' || cu.last_name as customer,
    COUNT(r.rental_id) AS total_rentals
FROM
    customer cu
    JOIN rental r ON r.customer_id = cu.customer_id
GROUP BY
    cu.first_name,
    cu.last_name
ORDER BY
    total_rentals DESC
LIMIT
    10;
""").df()

,customer,total_rentals
0,ELEANOR HUNT,46
1,KARL SEAL,45
2,MARCIA DEAN,42
3,CLARA SHAW,42
4,TAMMY SANDERS,41
5,SUE PETERS,40
6,WESLEY BULL,40
7,TIM CARY,39
8,MARION SNYDER,39
9,RHONDA KENNEDY,39


In [10]:
per_month = conn.sql("""--sql
SELECT
    DATE_TRUNC ('month', rental_date) AS rental_month,
    cu.first_name || ' ' || cu.last_name AS customer,
    COUNT(*) AS monthly_rentals
FROM
    rental r
    JOIN customer cu ON cu.customer_id = r.customer_id
GROUP BY
    rental_month,
    customer
""").df()

per_month.sample(10)

,rental_month,customer,monthly_rentals
1242,2006-02-01,MELVIN ELLINGTON,1
1536,2005-07-01,CONSTANCE REID,14
1351,2005-07-01,ENRIQUE FORSYTHE,6
2070,2005-07-01,PHILLIP HOLM,11
1491,2005-07-01,DIANNE SHELTON,9
1246,2005-05-01,GORDON ALLARD,1
917,2005-08-01,TERRENCE GUNDERSON,8
2101,2005-08-01,PAULA BRYANT,4
1628,2005-07-01,SARA PERRY,12
918,2005-08-01,MINNIE ROMERO,11


In [11]:
top_per_month = (
    per_month.sort_values(["rental_month", "monthly_rentals"], ascending=[True, False])
    .groupby("rental_month")
    .head(3)
)
top_per_month

,rental_month,customer,monthly_rentals
636,2005-05-01,SUE PETERS,8
643,2005-05-01,EDNA WEST,7
2296,2005-05-01,LESLIE SEWARD,7
214,2005-06-01,BRENDA WRIGHT,11
383,2005-06-01,ALEX GRESHAM,10
43,2005-06-01,MARGIE WADE,9
2395,2005-07-01,ELEANOR HUNT,22
557,2005-07-01,CRYSTAL FORD,21
1913,2005-07-01,TAMMY SANDERS,20
277,2005-08-01,SHERRY MARSHALL,18


#### f.3) Which films generate the highest total revenue?

In [12]:
conn.sql("""--sql
SELECT
    f.title,
    SUM(p.amount) AS total_amount
FROM
    film f
    JOIN inventory i ON i.film_id = f.film_id
    JOIN rental r ON r.inventory_id = i.inventory_id
    JOIN payment p ON p.rental_id = r.rental_id
GROUP BY
    f.title
ORDER BY
    total_amount DESC
LIMIT
    10;
        """).df()

,title,total_amount
0,TELEGRAPH VOYAGE,231.73
1,WIFE TURN,223.69
2,ZORRO ARK,214.69
3,GOODFELLAS SALUTE,209.69
4,SATURDAY LAMBS,204.72
5,TITANS JERK,201.71
6,TORQUE BOUND,198.72
7,HARRY IDAHO,195.70
8,INNOCENT USUAL,191.74
9,HUSTLER PARTY,190.78


#### f.4) Which categories have the longest average film length?

In [13]:
conn.sql("""--sql
SELECT
    c.name AS category,
    AVG(f.length)::INT AS average_movie_length
FROM
    category c
    JOIN film_category fc ON fc.category_id = c.category_id
    JOIN film f ON f.film_id = fc.film_id
GROUP BY
    category
ORDER BY
    average_movie_length DESC
LIMIT
    10;
    """).df()

,category,average_movie_length
0,Sports,128
1,Games,128
2,Foreign,122
3,Drama,121
4,Comedy,116
5,Family,115
6,Music,114
7,Travel,113
8,Action,112
9,Horror,112


#### f.5) Which store is no. 1 in rentals and revenue?

In [14]:
conn.sql("""--sql
SELECT
    s.store_id,
    COUNT(r.rental_id) as total_rental,
    SUM(p.amount) as total_revenue
FROM
    store s
    JOIN customer cu ON cu.store_id = s.store_id
    JOIN rental r ON r.customer_id = cu.customer_id
    JOIN payment p ON p.rental_id = r.rental_id
GROUP BY
    s.store_id
ORDER BY
    total_revenue DESC;
""").df()

,store_id,total_rental,total_revenue
0,1,8747,36997.53
1,2,7297,30409.03


In [15]:
conn.close()